# MNIST classification

In [37]:
import tensorflow as tf
import matplotlib
import kfp
from kfp.components import func_to_container_op, InputPath, OutputPath, create_component_from_func
from typing import NamedTuple
from kfp import dsl
import numpy as np
from typing import NamedTuple
from kubernetes import client as k8s_client

In [38]:
#with open('/tmp/krb5cc_1000', 'rb') as f:
    #print(f.read())
#    pass

In [39]:
def read_data() -> NamedTuple('results', [('data_dir', str)]):
    import tensorflow as tf
    import numpy as np
    from collections import namedtuple
    import os
    
    def setup_krb():
        import os
        krb_cred = b'\x05\x04\x00\x0c\x00\x01\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x07CERN.CH\x00\x00\x00\x06krbtgt\x00\x00\x00\x07CERN.CH\x00\x17\x00\x00\x00\x10\xb2\xf8&\xca\x05\x92\xd3\rp\x1dI\x19K\xdb\xee\xae_kH\x03_kH\x03_l\xa7\x90_q\xdf\x80\x00P\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tLa\x82\tH0\x82\tD\xa0\x03\x02\x01\x05\xa1\t\x1b\x07CERN.CH\xa2\x1c0\x1a\xa0\x03\x02\x01\x02\xa1\x130\x11\x1b\x06krbtgt\x1b\x07CERN.CH\xa3\x82\t\x120\x82\t\x0e\xa0\x03\x02\x01\x12\xa1\x03\x02\x01\x05\xa2\x82\t\x00\x04\x82\x08\xfc\xb0\xdb`\x15\x9d\xc8_\xcbi\x01\x02\x9e\xe4\xca\xce\xe1b\xa9?\xba\x93b9\xc3\xcf\x9a4\r\xb3\xf8\x8c\x8f\x15\x92O\x84\xda\xc1\x0fk\xc3\x00\x0b\xa7-\xd3d\x91\xde\xff\xa7\xcd\xf8\x06fv[\x127+\x0c\xfb\xde\xac\x8f\x979\x86\xcd\x89\x14-\x97\xae\xf7YkE\x02\xd9\x8d\xd2\xf3s\x0c\xee\x80_\xd7;O\x8e\xe9\xb3O\x1f\xd2\x03.\xe6e\x96\x8b9\xdd\xaf\xa1\xd0+\x99\x1d\xabq\'\xaf(\xce\xbc_\xa8\x8a\xe2\x85\x89l\xc4\x19\xb6q\x03\xfc)_\xa4\x90X\x7f\xcd\'\xd7\xe2\xcdGQt-9\xe6\xe2\r3`\x82\xce%\x02\xf8\x06\x05\x93\xd7b\x9d)\xab\xa5/\x00\xc4\x1e\x0b\xad\x14\xf5\r\xe6v\xfa\x17\'\xb2\xdb\xc4\xd4\x99\xf8\x02\x9e\xd7\xa3\xca\x88\xa0\x16\xc2\xb8u\x14h\xd3\x05(\x8a\xb4\x04\x9b\x91\xed\x06$\xa8\x8a\xc7AN\xb0\xfcGU\xae\xfd\xfa_\x02\xe0T\xb0\xf0k\xd9\x16%,9\x8d\x82\xbfrJGuJ\x01\x10[\x0f\xec\x1e\xab\x8b\x08\x96\xdcO:\xa9\xe4\xf9\x0c\xb2{?\xf7\xa4ig=%\x9e8\xde\xec\x1b\xce#\xf9\x92\xd09\xc5%\x86\xd4\xaa\xdbd0\xb5\x00![\xa9\xfe\x0e\xd6\x03\xcc\x97\xcb\xd3\xf3\xe0\x97\xd2\xfb\xbe\r]\x8c\xe4\x11\x0b\x87\xefyF3\x98\x98\xaa\x1a\x1dd\xed`?.\xd7a\xe2F&\x04\xb6\x88r\x0e\x97\xd7\x91\xb4;\xd2\x9b\xa7j\xed\x89?\x95\x9fB\xf7\xe6\'\xde\x8a\xa4\x1ew\xe0\x11yp<K\x87\x9f\xea\x15"\xa4Lk.\x8a\x1b\xc3MNv\xc5\xf5\x19\x03zP\xf6\x99\x13_\xa8*\xea\xbb\xbemA^f\x1dw\xf4*\x88!\xc4-\x904\xdfXm.\xfa\xdc\xe6\xc1,"g\xd0\xa4<\x1d\xa5\xd3\x1b\xd2)\xef>Q\x1aMV\xba\xcaRKW\xa9;\xd43Z\x9a\xf4\xf3?-\x93\xe9\x7f\xf6\xa5.cJR;\xad\xea\xd3q\xe2\xf4\x10\xf4\xeb}\xbb3\x07w\xeb\x87\xb6Cm\x86\xf8\xaef\xd4t\xa3r\n\x03\xfbZ\x95wM\xa9\x1d\x16\xb4\x03\xcc\xf0a\x12=;h\xd9\x074\xc8<~E\xac8\xf3\xf8\xbc\xe68\xf0\xc7i\x11Kwki\x12$\xdc7Y\xa6\x89\xca\x02\x81\x82a\x86\xaf{\xc4\xa41\xa11RR\xff\x14\xc6\xbf\x88\x1f\x7f3\xdf\x1c\x9cE\n\x97\xaa\xd1,\xb0\x13\xc1\xc77\xe0\x8a\xfe$\xff@\x015\x1c\xb4\xcfO\x1e\xfa\xaa\xa8y\x9e\x1db\x1c\xac]\xd1\xae\xf15\x9bR"\xc56_\x1a\x89Q<[\xd6\xf7}]\x0f\xdd{xQ\xfa\xa4R\xc2\x90\xb7\xcfI0!t\xd8tQ\x9f\x9d\x9f\x15c\xf4\xdf\xe3\xe5edY#\xd3\x89\xf8\xd1\xb97?\xd8K\x94\xeb\xe78\t?\xdb\xe2yZD"\xab\xf8\x0c\x86\x16\x07\xaf\xb1\xf2\x05\xa9V\x87\xb3p*\xac\x1c\x87N\xbb<\xeai\x90-uN@\x8a\xd5y\xdcl\xe5\xe4\x9b\xbf\xc1\xc7\xcf\xfcY\xdaPc\xe7D\xa8\xb6_5*\xf7Z}\x03H\xbd+;\xf2\xb6\xc5\xc4M \x07\xbfq\x8a\'\x07u\xe4\xf9\x9e\x89\xf8\xb6\xc8%\x06\xc0\xd0\x97\xa7\\1\xa1\xa2\xf7Oaf\xb2\x9aa\xdfU5L.\x19\x02M\xc1\xe5\xa9V\xd2\xef#mi\x93\x08\x14;K\x1c\x1a\xec\x81\xa2\x8d\xd4]\xd3}\xa2c\x05\xd1\r;\x14\x11\t\x18N\xa04\xb6{\x9d~4\xe2\xea\xcb\xc4\xacHW\xc6\x12a\xdd\xb9\xafG\x98v\xbbg76\xc6\xb9H\x19\xdc\x03I\xb6\x14\\\xdc\x89\xa6\x16\x068\xcfS\xa7\xc8\xe3s\x15\x8b\x12>\xb6Q\x16\x18\xa5\xe3~e\xeb\xfc\xe7\xbeP\x1b\xe2r\x12\xa1Z\xea/:p\xb5pW\x90\x0fOb]/\x80\x81\xfb\xee&{\xab\xf56\xbd\x1f\x01s\xea\xd2\t\x08\xc9\xa5\xc1P\xb5\x98\xbb\xfd#\xc8\xf0\x80\xa6\xc1\xb2\x1b\xe4\xb0[K\xfb\xd8\xfa\xb1\x8b\xb6EM\x80\rN\xd0p#]GX\xbb<\xa0G3\xb8h\xfc\x95Or\x80-\xdfK\x87\xdb\x9e\x13]\xfd\xe7\xa7\xd9\x13\xe9\'\x16qEq\xed\x91\xe1\xcc\x99\xa5\xa3a9\xf9\x88\x18\xdb\xbe\xaa\xff\xe2k\xcej\xa7\x87R\xd4\x00\xadO\xb6u\x0bW\x8f\xd0\xe5O\xd4\xe2\xc4W\x11g\xa8wU\xe3\xae\xff\xea\xc0- \xe1\xc8\xc3\x03/^\x94\n\xdcJ\xdd\x97[\xdbG\x9c\xfc\x0ezSd\xfe\x8d\x8c\x80\x9fHW"2\xe8\xf6pB\xc6\xe4ZWQ\xdd\xc8\xaf&\x91\x14]1\xcd\xe5\xfa{\xfc\xef\x01\xcb\xd7\xe1\xee[\xf3\xfe \xa2df\xc7B\xfa\xfeA\x80\xad\xc9X \x8c\x98s9/<QB\x1d\xb2\xc8\xbd3\x9f\xad\x07\x9bU?\n\x83\xb4\xcf)\xce\x85\x10LZ@`3\xceI\x07<\xa5\xb0\xf7\xe8EA\xe1\x9f\x04\x03\xa7\xaaZ2u\xfd\xd0m\xe6\xe7\xa5\xe7\x1f\x85\x13\x0f\xe6\x8a\x02\xb3\xb5\x9f1\x8a\xc6\x8a\x92\x91\xe6\xf0\xb4\xc8\xd4\x9d\xb1Y\xc7\'\xcb\x84\xfe\x92\xb1q\xce0\xa1\xfdj\xee\xa3\xd7\xe9\x02_\xcb\x1aS\xec\xefAq0\xd4n\xcf.\xf92\xd3\x06fcb\x96\xf4Ht\xf5[\xbd\xde\x06m\xc9\xd3\x93\xe9.\x1f!\x841\x05\xb3\x82l\x87\xec3c\x97\xf6d\xcb(0\xf5?\xcd\x1c\xe1!H\x1e\t\x14\x11\x19?\xbfG\x10\xb2*\xcb@\xbd_]VM\xdbP\x7f\xb2\x16\xb6/\x0cZs\x96\xcd_\xbd\xc2\xdc\x92po\x87\x8c\xcau\x9fD\x8b@\x0b.<\xfchr ]\x80GL \x8c\xb1.\xdd+\xb1\x88\x06\xd2\x96\xcd\xdeA\xee\xdfC\xd9\x99\x8c`-\x97Q\xa7\xd2\x05\xc4\x9b^b3\xa9\x9a\xfb\x01gr\\Y\xf0\xb8\xb4\x9c^\xcd`\x1c!q\x9f\x9b \xa35\xbc*\x1eP\xa4\xf4A\xa7u\x83G\xf0\xf41\x05\xd9w\xe2\xa6\x1a\x8c\xedo\xbf\xf6w&`\xff\xa9vC\xfeJ\xd4\x88\xd5P\x17!{\xef\xa0T\xd6\x17\x8ez?yZC/\xae.\x1a#\xbf\x9fV\x91T5\x14\x9aKH\x10V\x91(\xcd\x1a\xffPT\x8e\xdf\x8e\xb2\x05\xe6\xdb\xa9\xaf\xf5A&X\x96\x16\x0cF>\x15 \xe0I\x82\xc7\xa4\xca2\x85\x1f\xd4\x0b\xdc\xedL\xf8\xf4\xa6\x14\x0b\x11KvNY>\x1b\xaa<\x8c\xc1\x98\\2\xab\xe5[\xef\xf0\xca\x0b\xaa\xdf-i|\xfa\xebo@\xdb\xf1\xd5\xc9\xa1\xcf\xe6g\xa3\xdb\xcf\xf9\xc3\xb6=I\xb3U\x07<\xd3\xady\x89\xc9Xa\xed8\xd4\xadyj\xcf\xd9~\x7f\x13\x1c\x1b\xc3\xe9\xd8L\xc6\xdc\x11F\xbf\xdaX"6\xe2hD;\xc6wj\xa3\'\t\xf7\xe4#\xa8\xb6\xbe[/X\xcb\x96\rN\xe2\xd0\x1a\x88W\xff\x92;\x9e\x13\xf9*\xcc){Y01\xaa\xce\x127\xc0D5g\xa9\xcd8\x1d\x1f%$\x9a\x05k\xca\x9bQt\x17D\xfc?) F\x99[\x98\x12/-)\x94o9=@\xf1\x81\x1e\xfdK\xd2\xda\x86:\xbf\xe0\xcb#\x7f]MZ\r\xd0Y\x8ck\xd6ZU\xcb\xbe\xc5\xe4<\xcc~\x80\x8e\xfcv)\xc0-\x90\x927oRm\x1e\x0b\xac\x07\x17\x18\x98\xfc\xc8\x97\xa9\xc43\xfc[\x9b_\xd1\xc7\xcfPv-]\xe5{\xec0\xc0\x15}\xe0v\x81\x14\x9b\x1b\xc7|*\xaf+\xfd&\x1b\xfe\x03(R\'\xfc=\x8cSui$\xe1\xe1u\x92p\xc7&,\xfdW\xaf\x15$\xfcp\xcc\\\xcae|c\x88\xa2\x1a\xfc\xf2\xff\x9fC\xb1\xb2\xfd\x06F#~@LM\xa4\xdf=\x83\xa7e\xdc.~eY\xb8e\x08\xd1\xbd\xf7\x8cpT\xdb\xd7\x02\\C6U\\\x0b\xebO\xba;\xc7\x1d\xad\xda\xf8\xfa\xf5\xd9<*c\xfe\x90\xf7\xd4\x04\xb7\xcc\xf9\xfd\x8c\n\xf2\x85\x18l\x1e}9*QH\x95P\x8eSe\xb9\xa8\'\xc2\x0b\'\x88\x1a\xa0\x9f\x98_G\xa3\xbeE0\xd0\x8cX\xd0\x99\x0b\x17\x8f\xa9\r\xdd\xa7\xdd\x82+\xf7\x97\xf7\xc6,\xf0\xcf\xb3\xe7\xb1(\x11&\x8b\x00\x98kF\xdf\xc94\xfe:\x9a\x1b\xafx\x96\x90\xbc\x82\xfa\xe1\xf7\x0b\x95\x15\xbf_2H @"\x85\xdf\xbf\x7ff\xf7\xad,\xe8\x88\x85\x00\xb3=\n\x0eo\x87v\x99\xf2\x0e^w\xd0\xf4\xe6\xea\x08\xfb\xc0\x0c/2\x1a\xc8B\x04\x95\x91O\\K\xf8\xe1\x9f\xc7\x97\xf5\x17A\xd6\xf1T\xa7\x1b\x07z4\t\x91\x9d\x1f\x07\xc7\x19G\x05\x0c\rBO\xbd\xbc\xb9\x01\xa7sF\x01\xc0a\x1cSt\x99u\xcf"_hF!\x0f\xe6\x8aN"_\xaa\xc5\xf7o{0\xca*!Y\x19\x1c\x02\x96\x02\x9f\xd9\x99\xf8ZZK\xc5\xec\'\xf8#4\xab2\xde\xf0\x04\xd7\xde\xd1A\xdc\xac\xd4\xed\xa0\xd3\xb9\xb2\x8f\x99\xfa\xde\x90\xac\xf42\x93\xc8\x17J\xc3\x7fEr\xf7\x14P\x11\x11.\xfd\xe7T\xdf\x1d\xfe\xb3\xabV3\xc5\xb0\xdb\x9e\x05\xfc\x9dA\xb1\xb9\xe6\x01\x04\x13\xdfb\xc8\xf4S\xadV\x1e\xcb\xac\xed+\xf2c\xd3\xd8\xd5\xf4\n~V\xc04\x04\xc3\xe4\xf1\xbb\xb7\x12\xe2\x81\xd0\xfbY\x99\x98aZ\xfdGt\xb22\x0b\xac\x1c\xc0\x9e\xc1\xce\xed\xd3\x84\xaa\xfd\xb9\x17%\xed/\x01\x86\xb1\xca7\x13\x13=\x1d,\x07\x15\x04+?\x16\xcc#\x86U\x17\x7f\x97\xfe+\r5\x87t\xac\xd2?\x97\x9d\xb6)E\xb2\x01d=rlp\xce\xbf\x02\xb5?!\xb1\x02\xf7\xca\x03\xbd\xd4\x8b\xcf\xa1\xecq`\xa4\xee\xe0\xce\x08\xe30\xedw\xfc\xec\xef\x1e\xf4|\xe20\xa5>\x1b\x14\xfa\x8b\xc7G\x9f\xb7\xba\xa9\xcd"\xc0\x8e\x1c\x01\x00o\x99\xc9\xbb\xb7}\xca\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x0cX-CACHECONF:\x00\x00\x00\x15krb5_ccache_conf_data\x00\x00\x00\x07pa_type\x00\x00\x00\x16krbtgt/CERN.CH@CERN.CH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x012\x00\x00\x00\x00'
        with open('/tmp/krb5cc_1000', 'wb') as to_write:
            to_write.write(krb_cred)
        os.chmod("/tmp/krb5cc_1000", 0o600)
    setup_krb()
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

    data_dir = '/eos/user/d/dgolubov/demo-server-data/'
    
    np.save(data_dir + 'xtrain.npy', x_train)
    np.save(data_dir + 'ytrain.npy', y_train)
    
    np.save(data_dir + 'xtest.npy', x_test)
    np.save(data_dir + 'ytest.npy', y_test)
    
    result = namedtuple('results', ['data_dir'])
    return result(data_dir)

read_data_op = func_to_container_op(
        func=read_data, \
        base_image='gitlab-registry.cern.ch/ai-ml/images/tensorflow-2.1.0-notebook-gpu:1.0.0-2')

In [40]:
def preprocess_data(data_dir) -> NamedTuple('results', [('data_dir', str)]):
    import numpy as np
    import os
    from collections import namedtuple
    
    def setup_krb():
        import os
        krb_cred = b'\x05\x04\x00\x0c\x00\x01\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x07CERN.CH\x00\x00\x00\x06krbtgt\x00\x00\x00\x07CERN.CH\x00\x17\x00\x00\x00\x10\xb2\xf8&\xca\x05\x92\xd3\rp\x1dI\x19K\xdb\xee\xae_kH\x03_kH\x03_l\xa7\x90_q\xdf\x80\x00P\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tLa\x82\tH0\x82\tD\xa0\x03\x02\x01\x05\xa1\t\x1b\x07CERN.CH\xa2\x1c0\x1a\xa0\x03\x02\x01\x02\xa1\x130\x11\x1b\x06krbtgt\x1b\x07CERN.CH\xa3\x82\t\x120\x82\t\x0e\xa0\x03\x02\x01\x12\xa1\x03\x02\x01\x05\xa2\x82\t\x00\x04\x82\x08\xfc\xb0\xdb`\x15\x9d\xc8_\xcbi\x01\x02\x9e\xe4\xca\xce\xe1b\xa9?\xba\x93b9\xc3\xcf\x9a4\r\xb3\xf8\x8c\x8f\x15\x92O\x84\xda\xc1\x0fk\xc3\x00\x0b\xa7-\xd3d\x91\xde\xff\xa7\xcd\xf8\x06fv[\x127+\x0c\xfb\xde\xac\x8f\x979\x86\xcd\x89\x14-\x97\xae\xf7YkE\x02\xd9\x8d\xd2\xf3s\x0c\xee\x80_\xd7;O\x8e\xe9\xb3O\x1f\xd2\x03.\xe6e\x96\x8b9\xdd\xaf\xa1\xd0+\x99\x1d\xabq\'\xaf(\xce\xbc_\xa8\x8a\xe2\x85\x89l\xc4\x19\xb6q\x03\xfc)_\xa4\x90X\x7f\xcd\'\xd7\xe2\xcdGQt-9\xe6\xe2\r3`\x82\xce%\x02\xf8\x06\x05\x93\xd7b\x9d)\xab\xa5/\x00\xc4\x1e\x0b\xad\x14\xf5\r\xe6v\xfa\x17\'\xb2\xdb\xc4\xd4\x99\xf8\x02\x9e\xd7\xa3\xca\x88\xa0\x16\xc2\xb8u\x14h\xd3\x05(\x8a\xb4\x04\x9b\x91\xed\x06$\xa8\x8a\xc7AN\xb0\xfcGU\xae\xfd\xfa_\x02\xe0T\xb0\xf0k\xd9\x16%,9\x8d\x82\xbfrJGuJ\x01\x10[\x0f\xec\x1e\xab\x8b\x08\x96\xdcO:\xa9\xe4\xf9\x0c\xb2{?\xf7\xa4ig=%\x9e8\xde\xec\x1b\xce#\xf9\x92\xd09\xc5%\x86\xd4\xaa\xdbd0\xb5\x00![\xa9\xfe\x0e\xd6\x03\xcc\x97\xcb\xd3\xf3\xe0\x97\xd2\xfb\xbe\r]\x8c\xe4\x11\x0b\x87\xefyF3\x98\x98\xaa\x1a\x1dd\xed`?.\xd7a\xe2F&\x04\xb6\x88r\x0e\x97\xd7\x91\xb4;\xd2\x9b\xa7j\xed\x89?\x95\x9fB\xf7\xe6\'\xde\x8a\xa4\x1ew\xe0\x11yp<K\x87\x9f\xea\x15"\xa4Lk.\x8a\x1b\xc3MNv\xc5\xf5\x19\x03zP\xf6\x99\x13_\xa8*\xea\xbb\xbemA^f\x1dw\xf4*\x88!\xc4-\x904\xdfXm.\xfa\xdc\xe6\xc1,"g\xd0\xa4<\x1d\xa5\xd3\x1b\xd2)\xef>Q\x1aMV\xba\xcaRKW\xa9;\xd43Z\x9a\xf4\xf3?-\x93\xe9\x7f\xf6\xa5.cJR;\xad\xea\xd3q\xe2\xf4\x10\xf4\xeb}\xbb3\x07w\xeb\x87\xb6Cm\x86\xf8\xaef\xd4t\xa3r\n\x03\xfbZ\x95wM\xa9\x1d\x16\xb4\x03\xcc\xf0a\x12=;h\xd9\x074\xc8<~E\xac8\xf3\xf8\xbc\xe68\xf0\xc7i\x11Kwki\x12$\xdc7Y\xa6\x89\xca\x02\x81\x82a\x86\xaf{\xc4\xa41\xa11RR\xff\x14\xc6\xbf\x88\x1f\x7f3\xdf\x1c\x9cE\n\x97\xaa\xd1,\xb0\x13\xc1\xc77\xe0\x8a\xfe$\xff@\x015\x1c\xb4\xcfO\x1e\xfa\xaa\xa8y\x9e\x1db\x1c\xac]\xd1\xae\xf15\x9bR"\xc56_\x1a\x89Q<[\xd6\xf7}]\x0f\xdd{xQ\xfa\xa4R\xc2\x90\xb7\xcfI0!t\xd8tQ\x9f\x9d\x9f\x15c\xf4\xdf\xe3\xe5edY#\xd3\x89\xf8\xd1\xb97?\xd8K\x94\xeb\xe78\t?\xdb\xe2yZD"\xab\xf8\x0c\x86\x16\x07\xaf\xb1\xf2\x05\xa9V\x87\xb3p*\xac\x1c\x87N\xbb<\xeai\x90-uN@\x8a\xd5y\xdcl\xe5\xe4\x9b\xbf\xc1\xc7\xcf\xfcY\xdaPc\xe7D\xa8\xb6_5*\xf7Z}\x03H\xbd+;\xf2\xb6\xc5\xc4M \x07\xbfq\x8a\'\x07u\xe4\xf9\x9e\x89\xf8\xb6\xc8%\x06\xc0\xd0\x97\xa7\\1\xa1\xa2\xf7Oaf\xb2\x9aa\xdfU5L.\x19\x02M\xc1\xe5\xa9V\xd2\xef#mi\x93\x08\x14;K\x1c\x1a\xec\x81\xa2\x8d\xd4]\xd3}\xa2c\x05\xd1\r;\x14\x11\t\x18N\xa04\xb6{\x9d~4\xe2\xea\xcb\xc4\xacHW\xc6\x12a\xdd\xb9\xafG\x98v\xbbg76\xc6\xb9H\x19\xdc\x03I\xb6\x14\\\xdc\x89\xa6\x16\x068\xcfS\xa7\xc8\xe3s\x15\x8b\x12>\xb6Q\x16\x18\xa5\xe3~e\xeb\xfc\xe7\xbeP\x1b\xe2r\x12\xa1Z\xea/:p\xb5pW\x90\x0fOb]/\x80\x81\xfb\xee&{\xab\xf56\xbd\x1f\x01s\xea\xd2\t\x08\xc9\xa5\xc1P\xb5\x98\xbb\xfd#\xc8\xf0\x80\xa6\xc1\xb2\x1b\xe4\xb0[K\xfb\xd8\xfa\xb1\x8b\xb6EM\x80\rN\xd0p#]GX\xbb<\xa0G3\xb8h\xfc\x95Or\x80-\xdfK\x87\xdb\x9e\x13]\xfd\xe7\xa7\xd9\x13\xe9\'\x16qEq\xed\x91\xe1\xcc\x99\xa5\xa3a9\xf9\x88\x18\xdb\xbe\xaa\xff\xe2k\xcej\xa7\x87R\xd4\x00\xadO\xb6u\x0bW\x8f\xd0\xe5O\xd4\xe2\xc4W\x11g\xa8wU\xe3\xae\xff\xea\xc0- \xe1\xc8\xc3\x03/^\x94\n\xdcJ\xdd\x97[\xdbG\x9c\xfc\x0ezSd\xfe\x8d\x8c\x80\x9fHW"2\xe8\xf6pB\xc6\xe4ZWQ\xdd\xc8\xaf&\x91\x14]1\xcd\xe5\xfa{\xfc\xef\x01\xcb\xd7\xe1\xee[\xf3\xfe \xa2df\xc7B\xfa\xfeA\x80\xad\xc9X \x8c\x98s9/<QB\x1d\xb2\xc8\xbd3\x9f\xad\x07\x9bU?\n\x83\xb4\xcf)\xce\x85\x10LZ@`3\xceI\x07<\xa5\xb0\xf7\xe8EA\xe1\x9f\x04\x03\xa7\xaaZ2u\xfd\xd0m\xe6\xe7\xa5\xe7\x1f\x85\x13\x0f\xe6\x8a\x02\xb3\xb5\x9f1\x8a\xc6\x8a\x92\x91\xe6\xf0\xb4\xc8\xd4\x9d\xb1Y\xc7\'\xcb\x84\xfe\x92\xb1q\xce0\xa1\xfdj\xee\xa3\xd7\xe9\x02_\xcb\x1aS\xec\xefAq0\xd4n\xcf.\xf92\xd3\x06fcb\x96\xf4Ht\xf5[\xbd\xde\x06m\xc9\xd3\x93\xe9.\x1f!\x841\x05\xb3\x82l\x87\xec3c\x97\xf6d\xcb(0\xf5?\xcd\x1c\xe1!H\x1e\t\x14\x11\x19?\xbfG\x10\xb2*\xcb@\xbd_]VM\xdbP\x7f\xb2\x16\xb6/\x0cZs\x96\xcd_\xbd\xc2\xdc\x92po\x87\x8c\xcau\x9fD\x8b@\x0b.<\xfchr ]\x80GL \x8c\xb1.\xdd+\xb1\x88\x06\xd2\x96\xcd\xdeA\xee\xdfC\xd9\x99\x8c`-\x97Q\xa7\xd2\x05\xc4\x9b^b3\xa9\x9a\xfb\x01gr\\Y\xf0\xb8\xb4\x9c^\xcd`\x1c!q\x9f\x9b \xa35\xbc*\x1eP\xa4\xf4A\xa7u\x83G\xf0\xf41\x05\xd9w\xe2\xa6\x1a\x8c\xedo\xbf\xf6w&`\xff\xa9vC\xfeJ\xd4\x88\xd5P\x17!{\xef\xa0T\xd6\x17\x8ez?yZC/\xae.\x1a#\xbf\x9fV\x91T5\x14\x9aKH\x10V\x91(\xcd\x1a\xffPT\x8e\xdf\x8e\xb2\x05\xe6\xdb\xa9\xaf\xf5A&X\x96\x16\x0cF>\x15 \xe0I\x82\xc7\xa4\xca2\x85\x1f\xd4\x0b\xdc\xedL\xf8\xf4\xa6\x14\x0b\x11KvNY>\x1b\xaa<\x8c\xc1\x98\\2\xab\xe5[\xef\xf0\xca\x0b\xaa\xdf-i|\xfa\xebo@\xdb\xf1\xd5\xc9\xa1\xcf\xe6g\xa3\xdb\xcf\xf9\xc3\xb6=I\xb3U\x07<\xd3\xady\x89\xc9Xa\xed8\xd4\xadyj\xcf\xd9~\x7f\x13\x1c\x1b\xc3\xe9\xd8L\xc6\xdc\x11F\xbf\xdaX"6\xe2hD;\xc6wj\xa3\'\t\xf7\xe4#\xa8\xb6\xbe[/X\xcb\x96\rN\xe2\xd0\x1a\x88W\xff\x92;\x9e\x13\xf9*\xcc){Y01\xaa\xce\x127\xc0D5g\xa9\xcd8\x1d\x1f%$\x9a\x05k\xca\x9bQt\x17D\xfc?) F\x99[\x98\x12/-)\x94o9=@\xf1\x81\x1e\xfdK\xd2\xda\x86:\xbf\xe0\xcb#\x7f]MZ\r\xd0Y\x8ck\xd6ZU\xcb\xbe\xc5\xe4<\xcc~\x80\x8e\xfcv)\xc0-\x90\x927oRm\x1e\x0b\xac\x07\x17\x18\x98\xfc\xc8\x97\xa9\xc43\xfc[\x9b_\xd1\xc7\xcfPv-]\xe5{\xec0\xc0\x15}\xe0v\x81\x14\x9b\x1b\xc7|*\xaf+\xfd&\x1b\xfe\x03(R\'\xfc=\x8cSui$\xe1\xe1u\x92p\xc7&,\xfdW\xaf\x15$\xfcp\xcc\\\xcae|c\x88\xa2\x1a\xfc\xf2\xff\x9fC\xb1\xb2\xfd\x06F#~@LM\xa4\xdf=\x83\xa7e\xdc.~eY\xb8e\x08\xd1\xbd\xf7\x8cpT\xdb\xd7\x02\\C6U\\\x0b\xebO\xba;\xc7\x1d\xad\xda\xf8\xfa\xf5\xd9<*c\xfe\x90\xf7\xd4\x04\xb7\xcc\xf9\xfd\x8c\n\xf2\x85\x18l\x1e}9*QH\x95P\x8eSe\xb9\xa8\'\xc2\x0b\'\x88\x1a\xa0\x9f\x98_G\xa3\xbeE0\xd0\x8cX\xd0\x99\x0b\x17\x8f\xa9\r\xdd\xa7\xdd\x82+\xf7\x97\xf7\xc6,\xf0\xcf\xb3\xe7\xb1(\x11&\x8b\x00\x98kF\xdf\xc94\xfe:\x9a\x1b\xafx\x96\x90\xbc\x82\xfa\xe1\xf7\x0b\x95\x15\xbf_2H @"\x85\xdf\xbf\x7ff\xf7\xad,\xe8\x88\x85\x00\xb3=\n\x0eo\x87v\x99\xf2\x0e^w\xd0\xf4\xe6\xea\x08\xfb\xc0\x0c/2\x1a\xc8B\x04\x95\x91O\\K\xf8\xe1\x9f\xc7\x97\xf5\x17A\xd6\xf1T\xa7\x1b\x07z4\t\x91\x9d\x1f\x07\xc7\x19G\x05\x0c\rBO\xbd\xbc\xb9\x01\xa7sF\x01\xc0a\x1cSt\x99u\xcf"_hF!\x0f\xe6\x8aN"_\xaa\xc5\xf7o{0\xca*!Y\x19\x1c\x02\x96\x02\x9f\xd9\x99\xf8ZZK\xc5\xec\'\xf8#4\xab2\xde\xf0\x04\xd7\xde\xd1A\xdc\xac\xd4\xed\xa0\xd3\xb9\xb2\x8f\x99\xfa\xde\x90\xac\xf42\x93\xc8\x17J\xc3\x7fEr\xf7\x14P\x11\x11.\xfd\xe7T\xdf\x1d\xfe\xb3\xabV3\xc5\xb0\xdb\x9e\x05\xfc\x9dA\xb1\xb9\xe6\x01\x04\x13\xdfb\xc8\xf4S\xadV\x1e\xcb\xac\xed+\xf2c\xd3\xd8\xd5\xf4\n~V\xc04\x04\xc3\xe4\xf1\xbb\xb7\x12\xe2\x81\xd0\xfbY\x99\x98aZ\xfdGt\xb22\x0b\xac\x1c\xc0\x9e\xc1\xce\xed\xd3\x84\xaa\xfd\xb9\x17%\xed/\x01\x86\xb1\xca7\x13\x13=\x1d,\x07\x15\x04+?\x16\xcc#\x86U\x17\x7f\x97\xfe+\r5\x87t\xac\xd2?\x97\x9d\xb6)E\xb2\x01d=rlp\xce\xbf\x02\xb5?!\xb1\x02\xf7\xca\x03\xbd\xd4\x8b\xcf\xa1\xecq`\xa4\xee\xe0\xce\x08\xe30\xedw\xfc\xec\xef\x1e\xf4|\xe20\xa5>\x1b\x14\xfa\x8b\xc7G\x9f\xb7\xba\xa9\xcd"\xc0\x8e\x1c\x01\x00o\x99\xc9\xbb\xb7}\xca\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x0cX-CACHECONF:\x00\x00\x00\x15krb5_ccache_conf_data\x00\x00\x00\x07pa_type\x00\x00\x00\x16krbtgt/CERN.CH@CERN.CH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x012\x00\x00\x00\x00'
        with open('/tmp/krb5cc_1000', 'wb') as to_write:
            to_write.write(krb_cred)
        os.chmod("/tmp/krb5cc_1000", 0o600)
    setup_krb()
    
    x_train = np.load(data_dir + 'xtrain.npy')
    y_train = np.load(data_dir + 'ytrain.npy')

    x_test = np.load(data_dir + 'xtest.npy')
    y_test = np.load(data_dir + 'ytest.npy')
    
    # Filter 3 and 6
    def filter_36(x, y):
        keep = (y == 3) | (y == 6)
        x, y = x[keep], y[keep]
        y = y == 3
        return x,y
    
    print("Number of unfiltered training examples:", len(x_train))
    print("Number of unfiltered test examples:", len(x_test))
    
    x_train, y_train = filter_36(x_train, y_train)
    x_test, y_test = filter_36(x_test, y_test)

    print("Number of filtered training examples:", len(x_train))
    print("Number of filtered test examples:", len(x_test))
    
    np.save(data_dir + 'xtrain_filtered.npy', x_train)
    np.save(data_dir + 'ytrain_filtered.npy', y_train)
    
    np.save(data_dir + 'xtest_filtered.npy', x_test)
    np.save(data_dir + 'ytest_filtered.npy', y_test)
    
    result = namedtuple('results', ['data_dir'])
    return result(data_dir)

preprocess_data_op = func_to_container_op(func=preprocess_data, \
            base_image='gitlab-registry.cern.ch/ai-ml/images/tensorflow-2.1.0-notebook-gpu:1.0.0-2')

In [41]:
def model_full(data_dir) -> NamedTuple('results', [('result_path', str)]):
    import tensorflow as tf
    import numpy as np
    from collections import namedtuple
    
    def setup_krb():
        import os
        krb_cred = b'\x05\x04\x00\x0c\x00\x01\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x07CERN.CH\x00\x00\x00\x06krbtgt\x00\x00\x00\x07CERN.CH\x00\x17\x00\x00\x00\x10\xb2\xf8&\xca\x05\x92\xd3\rp\x1dI\x19K\xdb\xee\xae_kH\x03_kH\x03_l\xa7\x90_q\xdf\x80\x00P\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tLa\x82\tH0\x82\tD\xa0\x03\x02\x01\x05\xa1\t\x1b\x07CERN.CH\xa2\x1c0\x1a\xa0\x03\x02\x01\x02\xa1\x130\x11\x1b\x06krbtgt\x1b\x07CERN.CH\xa3\x82\t\x120\x82\t\x0e\xa0\x03\x02\x01\x12\xa1\x03\x02\x01\x05\xa2\x82\t\x00\x04\x82\x08\xfc\xb0\xdb`\x15\x9d\xc8_\xcbi\x01\x02\x9e\xe4\xca\xce\xe1b\xa9?\xba\x93b9\xc3\xcf\x9a4\r\xb3\xf8\x8c\x8f\x15\x92O\x84\xda\xc1\x0fk\xc3\x00\x0b\xa7-\xd3d\x91\xde\xff\xa7\xcd\xf8\x06fv[\x127+\x0c\xfb\xde\xac\x8f\x979\x86\xcd\x89\x14-\x97\xae\xf7YkE\x02\xd9\x8d\xd2\xf3s\x0c\xee\x80_\xd7;O\x8e\xe9\xb3O\x1f\xd2\x03.\xe6e\x96\x8b9\xdd\xaf\xa1\xd0+\x99\x1d\xabq\'\xaf(\xce\xbc_\xa8\x8a\xe2\x85\x89l\xc4\x19\xb6q\x03\xfc)_\xa4\x90X\x7f\xcd\'\xd7\xe2\xcdGQt-9\xe6\xe2\r3`\x82\xce%\x02\xf8\x06\x05\x93\xd7b\x9d)\xab\xa5/\x00\xc4\x1e\x0b\xad\x14\xf5\r\xe6v\xfa\x17\'\xb2\xdb\xc4\xd4\x99\xf8\x02\x9e\xd7\xa3\xca\x88\xa0\x16\xc2\xb8u\x14h\xd3\x05(\x8a\xb4\x04\x9b\x91\xed\x06$\xa8\x8a\xc7AN\xb0\xfcGU\xae\xfd\xfa_\x02\xe0T\xb0\xf0k\xd9\x16%,9\x8d\x82\xbfrJGuJ\x01\x10[\x0f\xec\x1e\xab\x8b\x08\x96\xdcO:\xa9\xe4\xf9\x0c\xb2{?\xf7\xa4ig=%\x9e8\xde\xec\x1b\xce#\xf9\x92\xd09\xc5%\x86\xd4\xaa\xdbd0\xb5\x00![\xa9\xfe\x0e\xd6\x03\xcc\x97\xcb\xd3\xf3\xe0\x97\xd2\xfb\xbe\r]\x8c\xe4\x11\x0b\x87\xefyF3\x98\x98\xaa\x1a\x1dd\xed`?.\xd7a\xe2F&\x04\xb6\x88r\x0e\x97\xd7\x91\xb4;\xd2\x9b\xa7j\xed\x89?\x95\x9fB\xf7\xe6\'\xde\x8a\xa4\x1ew\xe0\x11yp<K\x87\x9f\xea\x15"\xa4Lk.\x8a\x1b\xc3MNv\xc5\xf5\x19\x03zP\xf6\x99\x13_\xa8*\xea\xbb\xbemA^f\x1dw\xf4*\x88!\xc4-\x904\xdfXm.\xfa\xdc\xe6\xc1,"g\xd0\xa4<\x1d\xa5\xd3\x1b\xd2)\xef>Q\x1aMV\xba\xcaRKW\xa9;\xd43Z\x9a\xf4\xf3?-\x93\xe9\x7f\xf6\xa5.cJR;\xad\xea\xd3q\xe2\xf4\x10\xf4\xeb}\xbb3\x07w\xeb\x87\xb6Cm\x86\xf8\xaef\xd4t\xa3r\n\x03\xfbZ\x95wM\xa9\x1d\x16\xb4\x03\xcc\xf0a\x12=;h\xd9\x074\xc8<~E\xac8\xf3\xf8\xbc\xe68\xf0\xc7i\x11Kwki\x12$\xdc7Y\xa6\x89\xca\x02\x81\x82a\x86\xaf{\xc4\xa41\xa11RR\xff\x14\xc6\xbf\x88\x1f\x7f3\xdf\x1c\x9cE\n\x97\xaa\xd1,\xb0\x13\xc1\xc77\xe0\x8a\xfe$\xff@\x015\x1c\xb4\xcfO\x1e\xfa\xaa\xa8y\x9e\x1db\x1c\xac]\xd1\xae\xf15\x9bR"\xc56_\x1a\x89Q<[\xd6\xf7}]\x0f\xdd{xQ\xfa\xa4R\xc2\x90\xb7\xcfI0!t\xd8tQ\x9f\x9d\x9f\x15c\xf4\xdf\xe3\xe5edY#\xd3\x89\xf8\xd1\xb97?\xd8K\x94\xeb\xe78\t?\xdb\xe2yZD"\xab\xf8\x0c\x86\x16\x07\xaf\xb1\xf2\x05\xa9V\x87\xb3p*\xac\x1c\x87N\xbb<\xeai\x90-uN@\x8a\xd5y\xdcl\xe5\xe4\x9b\xbf\xc1\xc7\xcf\xfcY\xdaPc\xe7D\xa8\xb6_5*\xf7Z}\x03H\xbd+;\xf2\xb6\xc5\xc4M \x07\xbfq\x8a\'\x07u\xe4\xf9\x9e\x89\xf8\xb6\xc8%\x06\xc0\xd0\x97\xa7\\1\xa1\xa2\xf7Oaf\xb2\x9aa\xdfU5L.\x19\x02M\xc1\xe5\xa9V\xd2\xef#mi\x93\x08\x14;K\x1c\x1a\xec\x81\xa2\x8d\xd4]\xd3}\xa2c\x05\xd1\r;\x14\x11\t\x18N\xa04\xb6{\x9d~4\xe2\xea\xcb\xc4\xacHW\xc6\x12a\xdd\xb9\xafG\x98v\xbbg76\xc6\xb9H\x19\xdc\x03I\xb6\x14\\\xdc\x89\xa6\x16\x068\xcfS\xa7\xc8\xe3s\x15\x8b\x12>\xb6Q\x16\x18\xa5\xe3~e\xeb\xfc\xe7\xbeP\x1b\xe2r\x12\xa1Z\xea/:p\xb5pW\x90\x0fOb]/\x80\x81\xfb\xee&{\xab\xf56\xbd\x1f\x01s\xea\xd2\t\x08\xc9\xa5\xc1P\xb5\x98\xbb\xfd#\xc8\xf0\x80\xa6\xc1\xb2\x1b\xe4\xb0[K\xfb\xd8\xfa\xb1\x8b\xb6EM\x80\rN\xd0p#]GX\xbb<\xa0G3\xb8h\xfc\x95Or\x80-\xdfK\x87\xdb\x9e\x13]\xfd\xe7\xa7\xd9\x13\xe9\'\x16qEq\xed\x91\xe1\xcc\x99\xa5\xa3a9\xf9\x88\x18\xdb\xbe\xaa\xff\xe2k\xcej\xa7\x87R\xd4\x00\xadO\xb6u\x0bW\x8f\xd0\xe5O\xd4\xe2\xc4W\x11g\xa8wU\xe3\xae\xff\xea\xc0- \xe1\xc8\xc3\x03/^\x94\n\xdcJ\xdd\x97[\xdbG\x9c\xfc\x0ezSd\xfe\x8d\x8c\x80\x9fHW"2\xe8\xf6pB\xc6\xe4ZWQ\xdd\xc8\xaf&\x91\x14]1\xcd\xe5\xfa{\xfc\xef\x01\xcb\xd7\xe1\xee[\xf3\xfe \xa2df\xc7B\xfa\xfeA\x80\xad\xc9X \x8c\x98s9/<QB\x1d\xb2\xc8\xbd3\x9f\xad\x07\x9bU?\n\x83\xb4\xcf)\xce\x85\x10LZ@`3\xceI\x07<\xa5\xb0\xf7\xe8EA\xe1\x9f\x04\x03\xa7\xaaZ2u\xfd\xd0m\xe6\xe7\xa5\xe7\x1f\x85\x13\x0f\xe6\x8a\x02\xb3\xb5\x9f1\x8a\xc6\x8a\x92\x91\xe6\xf0\xb4\xc8\xd4\x9d\xb1Y\xc7\'\xcb\x84\xfe\x92\xb1q\xce0\xa1\xfdj\xee\xa3\xd7\xe9\x02_\xcb\x1aS\xec\xefAq0\xd4n\xcf.\xf92\xd3\x06fcb\x96\xf4Ht\xf5[\xbd\xde\x06m\xc9\xd3\x93\xe9.\x1f!\x841\x05\xb3\x82l\x87\xec3c\x97\xf6d\xcb(0\xf5?\xcd\x1c\xe1!H\x1e\t\x14\x11\x19?\xbfG\x10\xb2*\xcb@\xbd_]VM\xdbP\x7f\xb2\x16\xb6/\x0cZs\x96\xcd_\xbd\xc2\xdc\x92po\x87\x8c\xcau\x9fD\x8b@\x0b.<\xfchr ]\x80GL \x8c\xb1.\xdd+\xb1\x88\x06\xd2\x96\xcd\xdeA\xee\xdfC\xd9\x99\x8c`-\x97Q\xa7\xd2\x05\xc4\x9b^b3\xa9\x9a\xfb\x01gr\\Y\xf0\xb8\xb4\x9c^\xcd`\x1c!q\x9f\x9b \xa35\xbc*\x1eP\xa4\xf4A\xa7u\x83G\xf0\xf41\x05\xd9w\xe2\xa6\x1a\x8c\xedo\xbf\xf6w&`\xff\xa9vC\xfeJ\xd4\x88\xd5P\x17!{\xef\xa0T\xd6\x17\x8ez?yZC/\xae.\x1a#\xbf\x9fV\x91T5\x14\x9aKH\x10V\x91(\xcd\x1a\xffPT\x8e\xdf\x8e\xb2\x05\xe6\xdb\xa9\xaf\xf5A&X\x96\x16\x0cF>\x15 \xe0I\x82\xc7\xa4\xca2\x85\x1f\xd4\x0b\xdc\xedL\xf8\xf4\xa6\x14\x0b\x11KvNY>\x1b\xaa<\x8c\xc1\x98\\2\xab\xe5[\xef\xf0\xca\x0b\xaa\xdf-i|\xfa\xebo@\xdb\xf1\xd5\xc9\xa1\xcf\xe6g\xa3\xdb\xcf\xf9\xc3\xb6=I\xb3U\x07<\xd3\xady\x89\xc9Xa\xed8\xd4\xadyj\xcf\xd9~\x7f\x13\x1c\x1b\xc3\xe9\xd8L\xc6\xdc\x11F\xbf\xdaX"6\xe2hD;\xc6wj\xa3\'\t\xf7\xe4#\xa8\xb6\xbe[/X\xcb\x96\rN\xe2\xd0\x1a\x88W\xff\x92;\x9e\x13\xf9*\xcc){Y01\xaa\xce\x127\xc0D5g\xa9\xcd8\x1d\x1f%$\x9a\x05k\xca\x9bQt\x17D\xfc?) F\x99[\x98\x12/-)\x94o9=@\xf1\x81\x1e\xfdK\xd2\xda\x86:\xbf\xe0\xcb#\x7f]MZ\r\xd0Y\x8ck\xd6ZU\xcb\xbe\xc5\xe4<\xcc~\x80\x8e\xfcv)\xc0-\x90\x927oRm\x1e\x0b\xac\x07\x17\x18\x98\xfc\xc8\x97\xa9\xc43\xfc[\x9b_\xd1\xc7\xcfPv-]\xe5{\xec0\xc0\x15}\xe0v\x81\x14\x9b\x1b\xc7|*\xaf+\xfd&\x1b\xfe\x03(R\'\xfc=\x8cSui$\xe1\xe1u\x92p\xc7&,\xfdW\xaf\x15$\xfcp\xcc\\\xcae|c\x88\xa2\x1a\xfc\xf2\xff\x9fC\xb1\xb2\xfd\x06F#~@LM\xa4\xdf=\x83\xa7e\xdc.~eY\xb8e\x08\xd1\xbd\xf7\x8cpT\xdb\xd7\x02\\C6U\\\x0b\xebO\xba;\xc7\x1d\xad\xda\xf8\xfa\xf5\xd9<*c\xfe\x90\xf7\xd4\x04\xb7\xcc\xf9\xfd\x8c\n\xf2\x85\x18l\x1e}9*QH\x95P\x8eSe\xb9\xa8\'\xc2\x0b\'\x88\x1a\xa0\x9f\x98_G\xa3\xbeE0\xd0\x8cX\xd0\x99\x0b\x17\x8f\xa9\r\xdd\xa7\xdd\x82+\xf7\x97\xf7\xc6,\xf0\xcf\xb3\xe7\xb1(\x11&\x8b\x00\x98kF\xdf\xc94\xfe:\x9a\x1b\xafx\x96\x90\xbc\x82\xfa\xe1\xf7\x0b\x95\x15\xbf_2H @"\x85\xdf\xbf\x7ff\xf7\xad,\xe8\x88\x85\x00\xb3=\n\x0eo\x87v\x99\xf2\x0e^w\xd0\xf4\xe6\xea\x08\xfb\xc0\x0c/2\x1a\xc8B\x04\x95\x91O\\K\xf8\xe1\x9f\xc7\x97\xf5\x17A\xd6\xf1T\xa7\x1b\x07z4\t\x91\x9d\x1f\x07\xc7\x19G\x05\x0c\rBO\xbd\xbc\xb9\x01\xa7sF\x01\xc0a\x1cSt\x99u\xcf"_hF!\x0f\xe6\x8aN"_\xaa\xc5\xf7o{0\xca*!Y\x19\x1c\x02\x96\x02\x9f\xd9\x99\xf8ZZK\xc5\xec\'\xf8#4\xab2\xde\xf0\x04\xd7\xde\xd1A\xdc\xac\xd4\xed\xa0\xd3\xb9\xb2\x8f\x99\xfa\xde\x90\xac\xf42\x93\xc8\x17J\xc3\x7fEr\xf7\x14P\x11\x11.\xfd\xe7T\xdf\x1d\xfe\xb3\xabV3\xc5\xb0\xdb\x9e\x05\xfc\x9dA\xb1\xb9\xe6\x01\x04\x13\xdfb\xc8\xf4S\xadV\x1e\xcb\xac\xed+\xf2c\xd3\xd8\xd5\xf4\n~V\xc04\x04\xc3\xe4\xf1\xbb\xb7\x12\xe2\x81\xd0\xfbY\x99\x98aZ\xfdGt\xb22\x0b\xac\x1c\xc0\x9e\xc1\xce\xed\xd3\x84\xaa\xfd\xb9\x17%\xed/\x01\x86\xb1\xca7\x13\x13=\x1d,\x07\x15\x04+?\x16\xcc#\x86U\x17\x7f\x97\xfe+\r5\x87t\xac\xd2?\x97\x9d\xb6)E\xb2\x01d=rlp\xce\xbf\x02\xb5?!\xb1\x02\xf7\xca\x03\xbd\xd4\x8b\xcf\xa1\xecq`\xa4\xee\xe0\xce\x08\xe30\xedw\xfc\xec\xef\x1e\xf4|\xe20\xa5>\x1b\x14\xfa\x8b\xc7G\x9f\xb7\xba\xa9\xcd"\xc0\x8e\x1c\x01\x00o\x99\xc9\xbb\xb7}\xca\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x0cX-CACHECONF:\x00\x00\x00\x15krb5_ccache_conf_data\x00\x00\x00\x07pa_type\x00\x00\x00\x16krbtgt/CERN.CH@CERN.CH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x012\x00\x00\x00\x00'
        with open('/tmp/krb5cc_1000', 'wb') as to_write:
            to_write.write(krb_cred)
        os.chmod("/tmp/krb5cc_1000", 0o600)
    setup_krb()
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(28,28,1)))
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    model.summary()
    
    # Load data
    x_train = np.load(data_dir + 'xtrain_filtered.npy')
    y_train = np.load(data_dir + 'ytrain_filtered.npy')

    x_test = np.load(data_dir + 'xtest_filtered.npy')
    y_test = np.load(data_dir + 'ytest_filtered.npy')
    
    model.fit(x_train, y_train, batch_size=128, epochs=1, verbose=1, validation_data=(x_test, y_test))

    cnn_results = model.evaluate(x_test, y_test)
    print(cnn_results)
    
    with open(data_dir + 'model_full_result.txt', 'w') as writer:
        writer.write(str(cnn_results) + '\n')
        
    result = namedtuple('results', ['result_path'])
    return result(data_dir + 'model_full_result.txt')

model_full_op = func_to_container_op(func=model_full, \
    base_image='gitlab-registry.cern.ch/ai-ml/images/tensorflow-2.1.0-notebook-gpu:1.0.0-2')

In [42]:
def model_reduced(data_dir) -> NamedTuple('results', [('result_path', str)]):
    import tensorflow as tf
    import numpy as np
    from collections import namedtuple
    
    def setup_krb():
        import os
        krb_cred = b'\x05\x04\x00\x0c\x00\x01\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x07CERN.CH\x00\x00\x00\x06krbtgt\x00\x00\x00\x07CERN.CH\x00\x17\x00\x00\x00\x10\xb2\xf8&\xca\x05\x92\xd3\rp\x1dI\x19K\xdb\xee\xae_kH\x03_kH\x03_l\xa7\x90_q\xdf\x80\x00P\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tLa\x82\tH0\x82\tD\xa0\x03\x02\x01\x05\xa1\t\x1b\x07CERN.CH\xa2\x1c0\x1a\xa0\x03\x02\x01\x02\xa1\x130\x11\x1b\x06krbtgt\x1b\x07CERN.CH\xa3\x82\t\x120\x82\t\x0e\xa0\x03\x02\x01\x12\xa1\x03\x02\x01\x05\xa2\x82\t\x00\x04\x82\x08\xfc\xb0\xdb`\x15\x9d\xc8_\xcbi\x01\x02\x9e\xe4\xca\xce\xe1b\xa9?\xba\x93b9\xc3\xcf\x9a4\r\xb3\xf8\x8c\x8f\x15\x92O\x84\xda\xc1\x0fk\xc3\x00\x0b\xa7-\xd3d\x91\xde\xff\xa7\xcd\xf8\x06fv[\x127+\x0c\xfb\xde\xac\x8f\x979\x86\xcd\x89\x14-\x97\xae\xf7YkE\x02\xd9\x8d\xd2\xf3s\x0c\xee\x80_\xd7;O\x8e\xe9\xb3O\x1f\xd2\x03.\xe6e\x96\x8b9\xdd\xaf\xa1\xd0+\x99\x1d\xabq\'\xaf(\xce\xbc_\xa8\x8a\xe2\x85\x89l\xc4\x19\xb6q\x03\xfc)_\xa4\x90X\x7f\xcd\'\xd7\xe2\xcdGQt-9\xe6\xe2\r3`\x82\xce%\x02\xf8\x06\x05\x93\xd7b\x9d)\xab\xa5/\x00\xc4\x1e\x0b\xad\x14\xf5\r\xe6v\xfa\x17\'\xb2\xdb\xc4\xd4\x99\xf8\x02\x9e\xd7\xa3\xca\x88\xa0\x16\xc2\xb8u\x14h\xd3\x05(\x8a\xb4\x04\x9b\x91\xed\x06$\xa8\x8a\xc7AN\xb0\xfcGU\xae\xfd\xfa_\x02\xe0T\xb0\xf0k\xd9\x16%,9\x8d\x82\xbfrJGuJ\x01\x10[\x0f\xec\x1e\xab\x8b\x08\x96\xdcO:\xa9\xe4\xf9\x0c\xb2{?\xf7\xa4ig=%\x9e8\xde\xec\x1b\xce#\xf9\x92\xd09\xc5%\x86\xd4\xaa\xdbd0\xb5\x00![\xa9\xfe\x0e\xd6\x03\xcc\x97\xcb\xd3\xf3\xe0\x97\xd2\xfb\xbe\r]\x8c\xe4\x11\x0b\x87\xefyF3\x98\x98\xaa\x1a\x1dd\xed`?.\xd7a\xe2F&\x04\xb6\x88r\x0e\x97\xd7\x91\xb4;\xd2\x9b\xa7j\xed\x89?\x95\x9fB\xf7\xe6\'\xde\x8a\xa4\x1ew\xe0\x11yp<K\x87\x9f\xea\x15"\xa4Lk.\x8a\x1b\xc3MNv\xc5\xf5\x19\x03zP\xf6\x99\x13_\xa8*\xea\xbb\xbemA^f\x1dw\xf4*\x88!\xc4-\x904\xdfXm.\xfa\xdc\xe6\xc1,"g\xd0\xa4<\x1d\xa5\xd3\x1b\xd2)\xef>Q\x1aMV\xba\xcaRKW\xa9;\xd43Z\x9a\xf4\xf3?-\x93\xe9\x7f\xf6\xa5.cJR;\xad\xea\xd3q\xe2\xf4\x10\xf4\xeb}\xbb3\x07w\xeb\x87\xb6Cm\x86\xf8\xaef\xd4t\xa3r\n\x03\xfbZ\x95wM\xa9\x1d\x16\xb4\x03\xcc\xf0a\x12=;h\xd9\x074\xc8<~E\xac8\xf3\xf8\xbc\xe68\xf0\xc7i\x11Kwki\x12$\xdc7Y\xa6\x89\xca\x02\x81\x82a\x86\xaf{\xc4\xa41\xa11RR\xff\x14\xc6\xbf\x88\x1f\x7f3\xdf\x1c\x9cE\n\x97\xaa\xd1,\xb0\x13\xc1\xc77\xe0\x8a\xfe$\xff@\x015\x1c\xb4\xcfO\x1e\xfa\xaa\xa8y\x9e\x1db\x1c\xac]\xd1\xae\xf15\x9bR"\xc56_\x1a\x89Q<[\xd6\xf7}]\x0f\xdd{xQ\xfa\xa4R\xc2\x90\xb7\xcfI0!t\xd8tQ\x9f\x9d\x9f\x15c\xf4\xdf\xe3\xe5edY#\xd3\x89\xf8\xd1\xb97?\xd8K\x94\xeb\xe78\t?\xdb\xe2yZD"\xab\xf8\x0c\x86\x16\x07\xaf\xb1\xf2\x05\xa9V\x87\xb3p*\xac\x1c\x87N\xbb<\xeai\x90-uN@\x8a\xd5y\xdcl\xe5\xe4\x9b\xbf\xc1\xc7\xcf\xfcY\xdaPc\xe7D\xa8\xb6_5*\xf7Z}\x03H\xbd+;\xf2\xb6\xc5\xc4M \x07\xbfq\x8a\'\x07u\xe4\xf9\x9e\x89\xf8\xb6\xc8%\x06\xc0\xd0\x97\xa7\\1\xa1\xa2\xf7Oaf\xb2\x9aa\xdfU5L.\x19\x02M\xc1\xe5\xa9V\xd2\xef#mi\x93\x08\x14;K\x1c\x1a\xec\x81\xa2\x8d\xd4]\xd3}\xa2c\x05\xd1\r;\x14\x11\t\x18N\xa04\xb6{\x9d~4\xe2\xea\xcb\xc4\xacHW\xc6\x12a\xdd\xb9\xafG\x98v\xbbg76\xc6\xb9H\x19\xdc\x03I\xb6\x14\\\xdc\x89\xa6\x16\x068\xcfS\xa7\xc8\xe3s\x15\x8b\x12>\xb6Q\x16\x18\xa5\xe3~e\xeb\xfc\xe7\xbeP\x1b\xe2r\x12\xa1Z\xea/:p\xb5pW\x90\x0fOb]/\x80\x81\xfb\xee&{\xab\xf56\xbd\x1f\x01s\xea\xd2\t\x08\xc9\xa5\xc1P\xb5\x98\xbb\xfd#\xc8\xf0\x80\xa6\xc1\xb2\x1b\xe4\xb0[K\xfb\xd8\xfa\xb1\x8b\xb6EM\x80\rN\xd0p#]GX\xbb<\xa0G3\xb8h\xfc\x95Or\x80-\xdfK\x87\xdb\x9e\x13]\xfd\xe7\xa7\xd9\x13\xe9\'\x16qEq\xed\x91\xe1\xcc\x99\xa5\xa3a9\xf9\x88\x18\xdb\xbe\xaa\xff\xe2k\xcej\xa7\x87R\xd4\x00\xadO\xb6u\x0bW\x8f\xd0\xe5O\xd4\xe2\xc4W\x11g\xa8wU\xe3\xae\xff\xea\xc0- \xe1\xc8\xc3\x03/^\x94\n\xdcJ\xdd\x97[\xdbG\x9c\xfc\x0ezSd\xfe\x8d\x8c\x80\x9fHW"2\xe8\xf6pB\xc6\xe4ZWQ\xdd\xc8\xaf&\x91\x14]1\xcd\xe5\xfa{\xfc\xef\x01\xcb\xd7\xe1\xee[\xf3\xfe \xa2df\xc7B\xfa\xfeA\x80\xad\xc9X \x8c\x98s9/<QB\x1d\xb2\xc8\xbd3\x9f\xad\x07\x9bU?\n\x83\xb4\xcf)\xce\x85\x10LZ@`3\xceI\x07<\xa5\xb0\xf7\xe8EA\xe1\x9f\x04\x03\xa7\xaaZ2u\xfd\xd0m\xe6\xe7\xa5\xe7\x1f\x85\x13\x0f\xe6\x8a\x02\xb3\xb5\x9f1\x8a\xc6\x8a\x92\x91\xe6\xf0\xb4\xc8\xd4\x9d\xb1Y\xc7\'\xcb\x84\xfe\x92\xb1q\xce0\xa1\xfdj\xee\xa3\xd7\xe9\x02_\xcb\x1aS\xec\xefAq0\xd4n\xcf.\xf92\xd3\x06fcb\x96\xf4Ht\xf5[\xbd\xde\x06m\xc9\xd3\x93\xe9.\x1f!\x841\x05\xb3\x82l\x87\xec3c\x97\xf6d\xcb(0\xf5?\xcd\x1c\xe1!H\x1e\t\x14\x11\x19?\xbfG\x10\xb2*\xcb@\xbd_]VM\xdbP\x7f\xb2\x16\xb6/\x0cZs\x96\xcd_\xbd\xc2\xdc\x92po\x87\x8c\xcau\x9fD\x8b@\x0b.<\xfchr ]\x80GL \x8c\xb1.\xdd+\xb1\x88\x06\xd2\x96\xcd\xdeA\xee\xdfC\xd9\x99\x8c`-\x97Q\xa7\xd2\x05\xc4\x9b^b3\xa9\x9a\xfb\x01gr\\Y\xf0\xb8\xb4\x9c^\xcd`\x1c!q\x9f\x9b \xa35\xbc*\x1eP\xa4\xf4A\xa7u\x83G\xf0\xf41\x05\xd9w\xe2\xa6\x1a\x8c\xedo\xbf\xf6w&`\xff\xa9vC\xfeJ\xd4\x88\xd5P\x17!{\xef\xa0T\xd6\x17\x8ez?yZC/\xae.\x1a#\xbf\x9fV\x91T5\x14\x9aKH\x10V\x91(\xcd\x1a\xffPT\x8e\xdf\x8e\xb2\x05\xe6\xdb\xa9\xaf\xf5A&X\x96\x16\x0cF>\x15 \xe0I\x82\xc7\xa4\xca2\x85\x1f\xd4\x0b\xdc\xedL\xf8\xf4\xa6\x14\x0b\x11KvNY>\x1b\xaa<\x8c\xc1\x98\\2\xab\xe5[\xef\xf0\xca\x0b\xaa\xdf-i|\xfa\xebo@\xdb\xf1\xd5\xc9\xa1\xcf\xe6g\xa3\xdb\xcf\xf9\xc3\xb6=I\xb3U\x07<\xd3\xady\x89\xc9Xa\xed8\xd4\xadyj\xcf\xd9~\x7f\x13\x1c\x1b\xc3\xe9\xd8L\xc6\xdc\x11F\xbf\xdaX"6\xe2hD;\xc6wj\xa3\'\t\xf7\xe4#\xa8\xb6\xbe[/X\xcb\x96\rN\xe2\xd0\x1a\x88W\xff\x92;\x9e\x13\xf9*\xcc){Y01\xaa\xce\x127\xc0D5g\xa9\xcd8\x1d\x1f%$\x9a\x05k\xca\x9bQt\x17D\xfc?) F\x99[\x98\x12/-)\x94o9=@\xf1\x81\x1e\xfdK\xd2\xda\x86:\xbf\xe0\xcb#\x7f]MZ\r\xd0Y\x8ck\xd6ZU\xcb\xbe\xc5\xe4<\xcc~\x80\x8e\xfcv)\xc0-\x90\x927oRm\x1e\x0b\xac\x07\x17\x18\x98\xfc\xc8\x97\xa9\xc43\xfc[\x9b_\xd1\xc7\xcfPv-]\xe5{\xec0\xc0\x15}\xe0v\x81\x14\x9b\x1b\xc7|*\xaf+\xfd&\x1b\xfe\x03(R\'\xfc=\x8cSui$\xe1\xe1u\x92p\xc7&,\xfdW\xaf\x15$\xfcp\xcc\\\xcae|c\x88\xa2\x1a\xfc\xf2\xff\x9fC\xb1\xb2\xfd\x06F#~@LM\xa4\xdf=\x83\xa7e\xdc.~eY\xb8e\x08\xd1\xbd\xf7\x8cpT\xdb\xd7\x02\\C6U\\\x0b\xebO\xba;\xc7\x1d\xad\xda\xf8\xfa\xf5\xd9<*c\xfe\x90\xf7\xd4\x04\xb7\xcc\xf9\xfd\x8c\n\xf2\x85\x18l\x1e}9*QH\x95P\x8eSe\xb9\xa8\'\xc2\x0b\'\x88\x1a\xa0\x9f\x98_G\xa3\xbeE0\xd0\x8cX\xd0\x99\x0b\x17\x8f\xa9\r\xdd\xa7\xdd\x82+\xf7\x97\xf7\xc6,\xf0\xcf\xb3\xe7\xb1(\x11&\x8b\x00\x98kF\xdf\xc94\xfe:\x9a\x1b\xafx\x96\x90\xbc\x82\xfa\xe1\xf7\x0b\x95\x15\xbf_2H @"\x85\xdf\xbf\x7ff\xf7\xad,\xe8\x88\x85\x00\xb3=\n\x0eo\x87v\x99\xf2\x0e^w\xd0\xf4\xe6\xea\x08\xfb\xc0\x0c/2\x1a\xc8B\x04\x95\x91O\\K\xf8\xe1\x9f\xc7\x97\xf5\x17A\xd6\xf1T\xa7\x1b\x07z4\t\x91\x9d\x1f\x07\xc7\x19G\x05\x0c\rBO\xbd\xbc\xb9\x01\xa7sF\x01\xc0a\x1cSt\x99u\xcf"_hF!\x0f\xe6\x8aN"_\xaa\xc5\xf7o{0\xca*!Y\x19\x1c\x02\x96\x02\x9f\xd9\x99\xf8ZZK\xc5\xec\'\xf8#4\xab2\xde\xf0\x04\xd7\xde\xd1A\xdc\xac\xd4\xed\xa0\xd3\xb9\xb2\x8f\x99\xfa\xde\x90\xac\xf42\x93\xc8\x17J\xc3\x7fEr\xf7\x14P\x11\x11.\xfd\xe7T\xdf\x1d\xfe\xb3\xabV3\xc5\xb0\xdb\x9e\x05\xfc\x9dA\xb1\xb9\xe6\x01\x04\x13\xdfb\xc8\xf4S\xadV\x1e\xcb\xac\xed+\xf2c\xd3\xd8\xd5\xf4\n~V\xc04\x04\xc3\xe4\xf1\xbb\xb7\x12\xe2\x81\xd0\xfbY\x99\x98aZ\xfdGt\xb22\x0b\xac\x1c\xc0\x9e\xc1\xce\xed\xd3\x84\xaa\xfd\xb9\x17%\xed/\x01\x86\xb1\xca7\x13\x13=\x1d,\x07\x15\x04+?\x16\xcc#\x86U\x17\x7f\x97\xfe+\r5\x87t\xac\xd2?\x97\x9d\xb6)E\xb2\x01d=rlp\xce\xbf\x02\xb5?!\xb1\x02\xf7\xca\x03\xbd\xd4\x8b\xcf\xa1\xecq`\xa4\xee\xe0\xce\x08\xe30\xedw\xfc\xec\xef\x1e\xf4|\xe20\xa5>\x1b\x14\xfa\x8b\xc7G\x9f\xb7\xba\xa9\xcd"\xc0\x8e\x1c\x01\x00o\x99\xc9\xbb\xb7}\xca\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x0cX-CACHECONF:\x00\x00\x00\x15krb5_ccache_conf_data\x00\x00\x00\x07pa_type\x00\x00\x00\x16krbtgt/CERN.CH@CERN.CH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x012\x00\x00\x00\x00'
        with open('/tmp/krb5cc_1000', 'wb') as to_write:
            to_write.write(krb_cred)
        os.chmod("/tmp/krb5cc_1000", 0o600)
    setup_krb()
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28,28,1)))
    model.add(tf.keras.layers.Dense(2, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

    model.summary()
    
    # Load data
    x_train = np.load(data_dir + 'xtrain_filtered.npy')
    y_train = np.load(data_dir + 'ytrain_filtered.npy')

    x_test = np.load(data_dir + 'xtest_filtered.npy')
    y_test = np.load(data_dir + 'ytest_filtered.npy')
    
    model.fit(x_train, y_train, batch_size=128, epochs=1, verbose=1, validation_data=(x_test, y_test))

    cnn_results = model.evaluate(x_test, y_test)
    print(cnn_results)
    
    with open(data_dir + 'model_reduced_result.txt', 'w') as writer:
        writer.write(str(cnn_results) + '\n')
        
    result = namedtuple('results', ['result_path'])
    return result(data_dir + 'model_reduced_result.txt')

model_reduced_op = func_to_container_op(func=model_reduced, \
    base_image='gitlab-registry.cern.ch/ai-ml/images/tensorflow-2.1.0-notebook-gpu:1.0.0-2')

In [43]:
def models_evaluate(model_full_result_path, model_reduced_result_path):
    def setup_krb():
        import os
        krb_cred = b'\x05\x04\x00\x0c\x00\x01\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x07CERN.CH\x00\x00\x00\x06krbtgt\x00\x00\x00\x07CERN.CH\x00\x17\x00\x00\x00\x10\xb2\xf8&\xca\x05\x92\xd3\rp\x1dI\x19K\xdb\xee\xae_kH\x03_kH\x03_l\xa7\x90_q\xdf\x80\x00P\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\tLa\x82\tH0\x82\tD\xa0\x03\x02\x01\x05\xa1\t\x1b\x07CERN.CH\xa2\x1c0\x1a\xa0\x03\x02\x01\x02\xa1\x130\x11\x1b\x06krbtgt\x1b\x07CERN.CH\xa3\x82\t\x120\x82\t\x0e\xa0\x03\x02\x01\x12\xa1\x03\x02\x01\x05\xa2\x82\t\x00\x04\x82\x08\xfc\xb0\xdb`\x15\x9d\xc8_\xcbi\x01\x02\x9e\xe4\xca\xce\xe1b\xa9?\xba\x93b9\xc3\xcf\x9a4\r\xb3\xf8\x8c\x8f\x15\x92O\x84\xda\xc1\x0fk\xc3\x00\x0b\xa7-\xd3d\x91\xde\xff\xa7\xcd\xf8\x06fv[\x127+\x0c\xfb\xde\xac\x8f\x979\x86\xcd\x89\x14-\x97\xae\xf7YkE\x02\xd9\x8d\xd2\xf3s\x0c\xee\x80_\xd7;O\x8e\xe9\xb3O\x1f\xd2\x03.\xe6e\x96\x8b9\xdd\xaf\xa1\xd0+\x99\x1d\xabq\'\xaf(\xce\xbc_\xa8\x8a\xe2\x85\x89l\xc4\x19\xb6q\x03\xfc)_\xa4\x90X\x7f\xcd\'\xd7\xe2\xcdGQt-9\xe6\xe2\r3`\x82\xce%\x02\xf8\x06\x05\x93\xd7b\x9d)\xab\xa5/\x00\xc4\x1e\x0b\xad\x14\xf5\r\xe6v\xfa\x17\'\xb2\xdb\xc4\xd4\x99\xf8\x02\x9e\xd7\xa3\xca\x88\xa0\x16\xc2\xb8u\x14h\xd3\x05(\x8a\xb4\x04\x9b\x91\xed\x06$\xa8\x8a\xc7AN\xb0\xfcGU\xae\xfd\xfa_\x02\xe0T\xb0\xf0k\xd9\x16%,9\x8d\x82\xbfrJGuJ\x01\x10[\x0f\xec\x1e\xab\x8b\x08\x96\xdcO:\xa9\xe4\xf9\x0c\xb2{?\xf7\xa4ig=%\x9e8\xde\xec\x1b\xce#\xf9\x92\xd09\xc5%\x86\xd4\xaa\xdbd0\xb5\x00![\xa9\xfe\x0e\xd6\x03\xcc\x97\xcb\xd3\xf3\xe0\x97\xd2\xfb\xbe\r]\x8c\xe4\x11\x0b\x87\xefyF3\x98\x98\xaa\x1a\x1dd\xed`?.\xd7a\xe2F&\x04\xb6\x88r\x0e\x97\xd7\x91\xb4;\xd2\x9b\xa7j\xed\x89?\x95\x9fB\xf7\xe6\'\xde\x8a\xa4\x1ew\xe0\x11yp<K\x87\x9f\xea\x15"\xa4Lk.\x8a\x1b\xc3MNv\xc5\xf5\x19\x03zP\xf6\x99\x13_\xa8*\xea\xbb\xbemA^f\x1dw\xf4*\x88!\xc4-\x904\xdfXm.\xfa\xdc\xe6\xc1,"g\xd0\xa4<\x1d\xa5\xd3\x1b\xd2)\xef>Q\x1aMV\xba\xcaRKW\xa9;\xd43Z\x9a\xf4\xf3?-\x93\xe9\x7f\xf6\xa5.cJR;\xad\xea\xd3q\xe2\xf4\x10\xf4\xeb}\xbb3\x07w\xeb\x87\xb6Cm\x86\xf8\xaef\xd4t\xa3r\n\x03\xfbZ\x95wM\xa9\x1d\x16\xb4\x03\xcc\xf0a\x12=;h\xd9\x074\xc8<~E\xac8\xf3\xf8\xbc\xe68\xf0\xc7i\x11Kwki\x12$\xdc7Y\xa6\x89\xca\x02\x81\x82a\x86\xaf{\xc4\xa41\xa11RR\xff\x14\xc6\xbf\x88\x1f\x7f3\xdf\x1c\x9cE\n\x97\xaa\xd1,\xb0\x13\xc1\xc77\xe0\x8a\xfe$\xff@\x015\x1c\xb4\xcfO\x1e\xfa\xaa\xa8y\x9e\x1db\x1c\xac]\xd1\xae\xf15\x9bR"\xc56_\x1a\x89Q<[\xd6\xf7}]\x0f\xdd{xQ\xfa\xa4R\xc2\x90\xb7\xcfI0!t\xd8tQ\x9f\x9d\x9f\x15c\xf4\xdf\xe3\xe5edY#\xd3\x89\xf8\xd1\xb97?\xd8K\x94\xeb\xe78\t?\xdb\xe2yZD"\xab\xf8\x0c\x86\x16\x07\xaf\xb1\xf2\x05\xa9V\x87\xb3p*\xac\x1c\x87N\xbb<\xeai\x90-uN@\x8a\xd5y\xdcl\xe5\xe4\x9b\xbf\xc1\xc7\xcf\xfcY\xdaPc\xe7D\xa8\xb6_5*\xf7Z}\x03H\xbd+;\xf2\xb6\xc5\xc4M \x07\xbfq\x8a\'\x07u\xe4\xf9\x9e\x89\xf8\xb6\xc8%\x06\xc0\xd0\x97\xa7\\1\xa1\xa2\xf7Oaf\xb2\x9aa\xdfU5L.\x19\x02M\xc1\xe5\xa9V\xd2\xef#mi\x93\x08\x14;K\x1c\x1a\xec\x81\xa2\x8d\xd4]\xd3}\xa2c\x05\xd1\r;\x14\x11\t\x18N\xa04\xb6{\x9d~4\xe2\xea\xcb\xc4\xacHW\xc6\x12a\xdd\xb9\xafG\x98v\xbbg76\xc6\xb9H\x19\xdc\x03I\xb6\x14\\\xdc\x89\xa6\x16\x068\xcfS\xa7\xc8\xe3s\x15\x8b\x12>\xb6Q\x16\x18\xa5\xe3~e\xeb\xfc\xe7\xbeP\x1b\xe2r\x12\xa1Z\xea/:p\xb5pW\x90\x0fOb]/\x80\x81\xfb\xee&{\xab\xf56\xbd\x1f\x01s\xea\xd2\t\x08\xc9\xa5\xc1P\xb5\x98\xbb\xfd#\xc8\xf0\x80\xa6\xc1\xb2\x1b\xe4\xb0[K\xfb\xd8\xfa\xb1\x8b\xb6EM\x80\rN\xd0p#]GX\xbb<\xa0G3\xb8h\xfc\x95Or\x80-\xdfK\x87\xdb\x9e\x13]\xfd\xe7\xa7\xd9\x13\xe9\'\x16qEq\xed\x91\xe1\xcc\x99\xa5\xa3a9\xf9\x88\x18\xdb\xbe\xaa\xff\xe2k\xcej\xa7\x87R\xd4\x00\xadO\xb6u\x0bW\x8f\xd0\xe5O\xd4\xe2\xc4W\x11g\xa8wU\xe3\xae\xff\xea\xc0- \xe1\xc8\xc3\x03/^\x94\n\xdcJ\xdd\x97[\xdbG\x9c\xfc\x0ezSd\xfe\x8d\x8c\x80\x9fHW"2\xe8\xf6pB\xc6\xe4ZWQ\xdd\xc8\xaf&\x91\x14]1\xcd\xe5\xfa{\xfc\xef\x01\xcb\xd7\xe1\xee[\xf3\xfe \xa2df\xc7B\xfa\xfeA\x80\xad\xc9X \x8c\x98s9/<QB\x1d\xb2\xc8\xbd3\x9f\xad\x07\x9bU?\n\x83\xb4\xcf)\xce\x85\x10LZ@`3\xceI\x07<\xa5\xb0\xf7\xe8EA\xe1\x9f\x04\x03\xa7\xaaZ2u\xfd\xd0m\xe6\xe7\xa5\xe7\x1f\x85\x13\x0f\xe6\x8a\x02\xb3\xb5\x9f1\x8a\xc6\x8a\x92\x91\xe6\xf0\xb4\xc8\xd4\x9d\xb1Y\xc7\'\xcb\x84\xfe\x92\xb1q\xce0\xa1\xfdj\xee\xa3\xd7\xe9\x02_\xcb\x1aS\xec\xefAq0\xd4n\xcf.\xf92\xd3\x06fcb\x96\xf4Ht\xf5[\xbd\xde\x06m\xc9\xd3\x93\xe9.\x1f!\x841\x05\xb3\x82l\x87\xec3c\x97\xf6d\xcb(0\xf5?\xcd\x1c\xe1!H\x1e\t\x14\x11\x19?\xbfG\x10\xb2*\xcb@\xbd_]VM\xdbP\x7f\xb2\x16\xb6/\x0cZs\x96\xcd_\xbd\xc2\xdc\x92po\x87\x8c\xcau\x9fD\x8b@\x0b.<\xfchr ]\x80GL \x8c\xb1.\xdd+\xb1\x88\x06\xd2\x96\xcd\xdeA\xee\xdfC\xd9\x99\x8c`-\x97Q\xa7\xd2\x05\xc4\x9b^b3\xa9\x9a\xfb\x01gr\\Y\xf0\xb8\xb4\x9c^\xcd`\x1c!q\x9f\x9b \xa35\xbc*\x1eP\xa4\xf4A\xa7u\x83G\xf0\xf41\x05\xd9w\xe2\xa6\x1a\x8c\xedo\xbf\xf6w&`\xff\xa9vC\xfeJ\xd4\x88\xd5P\x17!{\xef\xa0T\xd6\x17\x8ez?yZC/\xae.\x1a#\xbf\x9fV\x91T5\x14\x9aKH\x10V\x91(\xcd\x1a\xffPT\x8e\xdf\x8e\xb2\x05\xe6\xdb\xa9\xaf\xf5A&X\x96\x16\x0cF>\x15 \xe0I\x82\xc7\xa4\xca2\x85\x1f\xd4\x0b\xdc\xedL\xf8\xf4\xa6\x14\x0b\x11KvNY>\x1b\xaa<\x8c\xc1\x98\\2\xab\xe5[\xef\xf0\xca\x0b\xaa\xdf-i|\xfa\xebo@\xdb\xf1\xd5\xc9\xa1\xcf\xe6g\xa3\xdb\xcf\xf9\xc3\xb6=I\xb3U\x07<\xd3\xady\x89\xc9Xa\xed8\xd4\xadyj\xcf\xd9~\x7f\x13\x1c\x1b\xc3\xe9\xd8L\xc6\xdc\x11F\xbf\xdaX"6\xe2hD;\xc6wj\xa3\'\t\xf7\xe4#\xa8\xb6\xbe[/X\xcb\x96\rN\xe2\xd0\x1a\x88W\xff\x92;\x9e\x13\xf9*\xcc){Y01\xaa\xce\x127\xc0D5g\xa9\xcd8\x1d\x1f%$\x9a\x05k\xca\x9bQt\x17D\xfc?) F\x99[\x98\x12/-)\x94o9=@\xf1\x81\x1e\xfdK\xd2\xda\x86:\xbf\xe0\xcb#\x7f]MZ\r\xd0Y\x8ck\xd6ZU\xcb\xbe\xc5\xe4<\xcc~\x80\x8e\xfcv)\xc0-\x90\x927oRm\x1e\x0b\xac\x07\x17\x18\x98\xfc\xc8\x97\xa9\xc43\xfc[\x9b_\xd1\xc7\xcfPv-]\xe5{\xec0\xc0\x15}\xe0v\x81\x14\x9b\x1b\xc7|*\xaf+\xfd&\x1b\xfe\x03(R\'\xfc=\x8cSui$\xe1\xe1u\x92p\xc7&,\xfdW\xaf\x15$\xfcp\xcc\\\xcae|c\x88\xa2\x1a\xfc\xf2\xff\x9fC\xb1\xb2\xfd\x06F#~@LM\xa4\xdf=\x83\xa7e\xdc.~eY\xb8e\x08\xd1\xbd\xf7\x8cpT\xdb\xd7\x02\\C6U\\\x0b\xebO\xba;\xc7\x1d\xad\xda\xf8\xfa\xf5\xd9<*c\xfe\x90\xf7\xd4\x04\xb7\xcc\xf9\xfd\x8c\n\xf2\x85\x18l\x1e}9*QH\x95P\x8eSe\xb9\xa8\'\xc2\x0b\'\x88\x1a\xa0\x9f\x98_G\xa3\xbeE0\xd0\x8cX\xd0\x99\x0b\x17\x8f\xa9\r\xdd\xa7\xdd\x82+\xf7\x97\xf7\xc6,\xf0\xcf\xb3\xe7\xb1(\x11&\x8b\x00\x98kF\xdf\xc94\xfe:\x9a\x1b\xafx\x96\x90\xbc\x82\xfa\xe1\xf7\x0b\x95\x15\xbf_2H @"\x85\xdf\xbf\x7ff\xf7\xad,\xe8\x88\x85\x00\xb3=\n\x0eo\x87v\x99\xf2\x0e^w\xd0\xf4\xe6\xea\x08\xfb\xc0\x0c/2\x1a\xc8B\x04\x95\x91O\\K\xf8\xe1\x9f\xc7\x97\xf5\x17A\xd6\xf1T\xa7\x1b\x07z4\t\x91\x9d\x1f\x07\xc7\x19G\x05\x0c\rBO\xbd\xbc\xb9\x01\xa7sF\x01\xc0a\x1cSt\x99u\xcf"_hF!\x0f\xe6\x8aN"_\xaa\xc5\xf7o{0\xca*!Y\x19\x1c\x02\x96\x02\x9f\xd9\x99\xf8ZZK\xc5\xec\'\xf8#4\xab2\xde\xf0\x04\xd7\xde\xd1A\xdc\xac\xd4\xed\xa0\xd3\xb9\xb2\x8f\x99\xfa\xde\x90\xac\xf42\x93\xc8\x17J\xc3\x7fEr\xf7\x14P\x11\x11.\xfd\xe7T\xdf\x1d\xfe\xb3\xabV3\xc5\xb0\xdb\x9e\x05\xfc\x9dA\xb1\xb9\xe6\x01\x04\x13\xdfb\xc8\xf4S\xadV\x1e\xcb\xac\xed+\xf2c\xd3\xd8\xd5\xf4\n~V\xc04\x04\xc3\xe4\xf1\xbb\xb7\x12\xe2\x81\xd0\xfbY\x99\x98aZ\xfdGt\xb22\x0b\xac\x1c\xc0\x9e\xc1\xce\xed\xd3\x84\xaa\xfd\xb9\x17%\xed/\x01\x86\xb1\xca7\x13\x13=\x1d,\x07\x15\x04+?\x16\xcc#\x86U\x17\x7f\x97\xfe+\r5\x87t\xac\xd2?\x97\x9d\xb6)E\xb2\x01d=rlp\xce\xbf\x02\xb5?!\xb1\x02\xf7\xca\x03\xbd\xd4\x8b\xcf\xa1\xecq`\xa4\xee\xe0\xce\x08\xe30\xedw\xfc\xec\xef\x1e\xf4|\xe20\xa5>\x1b\x14\xfa\x8b\xc7G\x9f\xb7\xba\xa9\xcd"\xc0\x8e\x1c\x01\x00o\x99\xc9\xbb\xb7}\xca\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x07CERN.CH\x00\x00\x00\x08dgolubov\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x0cX-CACHECONF:\x00\x00\x00\x15krb5_ccache_conf_data\x00\x00\x00\x07pa_type\x00\x00\x00\x16krbtgt/CERN.CH@CERN.CH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x012\x00\x00\x00\x00'
        with open('/tmp/krb5cc_1000', 'wb') as to_write:
            to_write.write(krb_cred)
        os.chmod("/tmp/krb5cc_1000", 0o600)
    setup_krb()
    
    print('Full model results:')
    with open(model_full_result_path, 'r') as reader:
        for line in reader:
            print(line, end = '')
    print('\nReduced model results:')
    with open(model_reduced_result_path, 'r') as reader:
        for line in reader:
            print(line, end = '')
            
models_evaluate_op = func_to_container_op(func=models_evaluate, \
    base_image='gitlab-registry.cern.ch/ai-ml/images/tensorflow-2.1.0-notebook-gpu:1.0.0-2')

In [44]:
def pipeline_component_with_volume(pipeline_task):
    return pipeline_task.add_volume(k8s_client.V1Volume(name='eos', host_path=k8s_client.V1HostPathVolumeSource(\
        path='/var/eos'))).add_volume_mount(k8s_client.V1VolumeMount(mount_path='/eos',name='eos'))

@dsl.pipeline(
    name='ML first',
    description='ML first.'
)
def ml_pipeline_first():
    read_data_task = read_data_op()
    read_data_task = pipeline_component_with_volume(read_data_task)
    
    preprocess_data_task = preprocess_data_op(read_data_task.outputs['data_dir'])
    preprocess_data_task = pipeline_component_with_volume(preprocess_data_task)
    
    model_full_task = model_full_op(preprocess_data_task.outputs['data_dir'])
    model_full_task = pipeline_component_with_volume(model_full_task)
    
    model_reduced_task = model_reduced_op(preprocess_data_task.outputs['data_dir'])
    model_reduced_task = pipeline_component_with_volume(model_reduced_task)
    
    evaluate_models_task = models_evaluate_op(model_full_task.outputs['result_path'], \
                                              model_reduced_task.outputs['result_path'])
    evaluate_models_task = pipeline_component_with_volume(evaluate_models_task)
    

In [45]:
kfp.compiler.Compiler().compile(ml_pipeline_first, 'ml_pipeline_demo.yaml')